In [1]:
import numpy as np
import json
import os
from scipy.spatial import distance

from allennlp.common.file_utils import cached_path
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from scipy.spatial import distance
from sklearn.metrics.cluster import v_measure_score



import sys
sys.path.append('/home/alonj/contrastive')
from nullspace_projection.src.inlp_oop.counterfactual import generate_counterfactuals
import nullspace_projection.src.debias as debias
import torch

task1 = "bios"
task2 = "bios"
train_name = "train"
dev_name = "dev"
test_name="test"

model_path = f"../s3-link/experiments/models/{task1}/roberta-large"
train_encodings_normal = np.load(model_path + f"/encodings/predicted_{task2}_{train_name}_encoded_representations.npy")
# train_labels = np.load(model_path + f"/encodings/predicted_{task}_{train_name}_labels.npy")
train_preds_normal = np.load(model_path + f"/encodings/predicted_{task2}_{train_name}_predictions.npy")
enc_size = train_encodings_normal.shape[1]
print(enc_size)

dev_encodings_normal = np.load(model_path + f"/encodings/predicted_{task2}_{dev_name}_encoded_representations.npy")
# dev_labels = np.load(model_path + f"/encodings/predicted_{task}_{dev_name}_labels.npy")
dev_preds_normal = np.load(model_path + f"/encodings/predicted_{task2}_{dev_name}_predictions.npy")

test_encodings_normal = np.load(model_path + f"/encodings/predicted_{task2}_{test_name}_encoded_representations.npy")
# dev_labels = np.load(model_path + f"/encodings/predicted_{task}_{dev_name}_labels.npy")
test_preds_normal = np.load(model_path + f"/encodings/predicted_{task2}_{test_name}_predictions.npy")

w = np.load(model_path + "/w.npy")  # shape: (num_classes, enc_size)
b = np.load(model_path + "/b.npy")  # shape: (num_classes)
with open(model_path + "/label2index.json", "r") as f:
    label2index = json.load(f)
    index2label = {label2index[k]: k for k in label2index}
    
    

1024


In [2]:
train_path = "../data/bios/train.jsonl"
dev_path = "../data/bios/dev.jsonl"
test_path = "../data/bios/test.jsonl"

with open(cached_path(dev_path)) as f:
    dev_data = [json.loads(line) for line in f if line.strip() if line.strip()]
# print([d['gold_label'] for d in dev_data])
dev_labels = np.array([label2index[d['label']] for d in dev_data])

with open(cached_path(train_path)) as f:
    train_data = [json.loads(line) for line in f if line.strip() if line.strip()]
train_labels = np.array([label2index[d['label']] for d in train_data])

with open(cached_path(test_path)) as f:
    test_data = [json.loads(line) for line in f if line.strip() if line.strip()]
test_labels = np.array([label2index[d['label']] for d in test_data])


In [3]:
dev_data[0]

{'text': 'He is trained and board certified in Emergency Medicine and is a Fellow of the American College of Emergency Physicians. He is a graduate of Brown University School of Medicine and also has a Masters degree in Public Health from Harvard University. Dr. Antosia has extensive medical writing experience with numerous publications in peer-reviewed journals and textbooks, as well as prior international, public health, and disaster-related experience. He has also lectured at local, national, and international forums including England, Ireland and Italy. In addition, he has published numerous abstracts, original articles and authored several book chapters.',
 'original_text': 'Dr. Robert Antosia is an attending physician at Beth Israel Deaconess Medical Center in Boston, Massachusetts and an Assistant Professor at Harvard Medical School. He is trained and board certified in Emergency Medicine and is a Fellow of the American College of Emergency Physicians. He is a graduate of Brown 

In [33]:
index2label

{0: 'professor',
 1: 'physician',
 2: 'attorney',
 3: 'photographer',
 4: 'journalist',
 5: 'nurse',
 6: 'psychologist',
 7: 'teacher',
 8: 'surgeon',
 9: 'dentist',
 10: 'architect',
 11: 'painter',
 12: 'poet',
 13: 'filmmaker',
 14: 'software_engineer',
 15: 'composer',
 16: 'accountant',
 17: 'dietitian',
 18: 'comedian',
 19: 'pastor',
 20: 'chiropractor',
 21: 'yoga_teacher',
 22: 'paralegal',
 23: 'dj',
 24: 'interior_designer',
 25: 'personal_trainer',
 26: 'rapper'}

In [3]:

task2 = "bios"

train_encodings_irrelevant = np.load(model_path + f"/encodings/predicted_{task2}_{train_name}_highlight_encoded_representations.npy")
# train_labels = np.load(model_path + f"/encodings/predicted_{task}_{train_name}_labels.npy")
train_preds_irrelevant = np.load(model_path + f"/encodings/predicted_{task2}_{train_name}_highlight_predictions.npy")

dev_encodings_irrelevant = np.load(model_path + f"/encodings/predicted_{task2}_{dev_name}_highlight_encoded_representations.npy")
# dev_labels = np.load(model_path + f"/encodings/predicted_{task}_{dev_name}_labels.npy")
dev_preds_irrelevant = np.load(model_path + f"/encodings/predicted_{task2}_{dev_name}_highlight_predictions.npy")

test_encodings_irrelevant = np.load(model_path + f"/encodings/predicted_{task2}_{test_name}_highlight_encoded_representations.npy")
# dev_labels = np.load(model_path + f"/encodings/predicted_{task}_{dev_name}_labels.npy")
test_preds_irrelevant = np.load(model_path + f"/encodings/predicted_{task2}_{test_name}_highlight_predictions.npy")


In [4]:
def tsne(vecs, labels, title="", ind2label = None, words = None, metric = "l2"):

  #tsne = TSNE(n_components=2)#, angle = 0.5, perplexity = 20)
  tsne = PCA()
  vecs_2d = tsne.fit_transform(vecs)
  label_names = sorted(list(set(labels.tolist())))
  num_labels = len(label_names)

  names = sorted(set(labels.tolist()))

  plt.figure(figsize=(6, 5))

  colors = "red", "blue", "yellow"
  for i, c, label in zip(sorted(set(labels.tolist())), colors, names):
     plt.scatter(vecs_2d[labels == i, 0], vecs_2d[labels == i, 1], c=c, s=len(c) * 5 + 20,
                label=label if ind2label is None else ind2label[label], alpha = 0.2, marker = "s" if i==0 else "o")
     plt.legend(loc = "upper right")

  plt.title(title)
  plt.show()
  #return vecs_2d

# ig, axs = plt.subplots(2, 2)
# axs[0, 0].plot(x, y)
# axs[0, 0].set_title('Axis [0,0]')
# axs[0, 1].plot(x, y, 'tab:orange')
# axs[0, 1].set_title('Axis [0,1]')
# axs[1, 0].plot(x, -y, 'tab:green')
# axs[1, 0].set_title('Axis [1,0]')
# axs[1, 1].plot(x, -y, 'tab:red')
# axs[1, 1].set_title('Axis [1,1]')


In [41]:
def tsne_axis(vecs, labels, title="", ind2label = None, words = None, metric = "l2", axs=None):

  #tsne = TSNE(n_components=2)#, angle = 0.5, perplexity = 20)
  tsne = PCA()
  vecs_2d = tsne.fit_transform(vecs)
  label_names = sorted(list(set(labels.tolist())))
  num_labels = len(label_names)

  names = sorted(set(labels.tolist()))
    
  if axs is None:
      plt.figure(figsize=(6, 5))
      colors = "red", "blue", "yellow"
      for i, c, label in zip(sorted(set(labels.tolist())), colors, names):
         plt.scatter(vecs_2d[labels == i, 0], vecs_2d[labels == i, 1], c=c, s=len(c) * 5 + 20,
                    label=label if ind2label is None else ind2label[label], alpha = 0.2, marker = "s" if i==0 else "o")
         plt.legend(loc = "upper right")

      plt.title(title)
      plt.show()
  else:
      colors = "red", "blue", "yellow"
      for i, c, label in zip(sorted(set(labels.tolist())), colors, names):
         axs.scatter(vecs_2d[labels == i, 0], vecs_2d[labels == i, 1], c=c, s=len(c) * 5 + 20,
                    label=label if ind2label is None else ind2label[label], alpha = 0.2, marker = "s" if i==0 else "o")
         axs.legend(loc = "upper right")
      axs.set_title(title, fontsize=20)


  #return vecs_2d

# ig, axs = plt.subplots(2, 2)
# axs[0, 0].plot(x, y)
# axs[0, 0].set_title('Axis [0,0]')
# axs[0, 1].plot(x, y, 'tab:orange')
# axs[0, 1].set_title('Axis [0,1]')
# axs[1, 0].plot(x, -y, 'tab:green')
# axs[1, 0].set_title('Axis [1,0]')
# axs[1, 1].plot(x, -y, 'tab:red')
# axs[1, 1].set_title('Axis [1,1]')


In [4]:
from sklearn.metrics.pairwise import euclidean_distances
def cluster_distance(points, clusters):
    # assumes binary clustering
    
    c0_centroid = points[clusters == 0].mean(axis=0)
    c1_centroid = points[clusters == 1].mean(axis=0)
    return euclidean_distances([c0_centroid, c1_centroid])[0,1]

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import v_measure_score
def cluster_distance_2(points, clusters):
    kmeans = KMeans(n_clusters=2, random_state=0).fit(points).labels_
    return v_measure_score(clusters, kmeans)

In [5]:
index2label

{0: 'professor',
 1: 'physician',
 2: 'attorney',
 3: 'photographer',
 4: 'journalist',
 5: 'nurse',
 6: 'psychologist',
 7: 'teacher',
 8: 'surgeon',
 9: 'dentist',
 10: 'architect',
 11: 'painter',
 12: 'poet',
 13: 'filmmaker',
 14: 'software_engineer',
 15: 'composer',
 16: 'accountant',
 17: 'dietitian',
 18: 'comedian',
 19: 'pastor',
 20: 'chiropractor',
 21: 'yoga_teacher',
 22: 'paralegal',
 23: 'dj',
 24: 'interior_designer',
 25: 'personal_trainer',
 26: 'rapper'}

In [5]:
diffs = []
for fact in [10]:
    diffs.append([])
    for foil in index2label:
        if fact == foil:
            diffs[-1].append(-10000)
            continue

        ind2label = [index2label[fact], index2label[foil]]
        
        print(f"\t\t\t\t\t{index2label[fact]} vs {index2label[foil]}")
        
        num_classifiers = 'early'

        contrastive_projection = np.load(model_path + f"/projections/{num_classifiers}/behavioral/contrastive_{max(fact, foil)}_{min(fact, foil)}.npy")
#         ambiguous_projection = np.load(model_path + f"/projections/{num_classifiers}/ambiguous_{max(fact, foil)}_{min(fact, foil)}.npy")
#         classifiers = np.load(model_path + f"/projections/{num_classifiers}/classifiers_{max(fact, foil)}_{min(fact, foil)}.npy")
        contrastive_projection_nh = np.load(model_path + f"/projections/{num_classifiers}/behavioral/nh/contrastive_{max(fact, foil)}_{min(fact, foil)}.npy")
    
        contrastive_projection = debias.get_rowspace_projection((w[fact] - w[foil]).reshape(1, -1))
        contrastive_projection_nh = contrastive_projection
    
#         classifiers = classifiers.reshape(num_classifiers, enc_size)
        
        contrastive_filter = np.any([dev_preds_normal == fact, dev_preds_normal == foil], axis=0)
        contrastive_filter_nh = np.any([dev_preds_irrelevant == fact, dev_preds_irrelevant == foil], axis=0)
        
        contrastive_filter = np.any([dev_preds_normal == fact, dev_preds_irrelevant == foil], axis=0)
        contrastive_filter_nh = contrastive_filter
        
        z_all = dev_encodings_normal[contrastive_filter]
#         z_h = dev_encodings_highlight[contrastive_filter]
        z_nh = dev_encodings_irrelevant[contrastive_filter_nh]
        z_all_row = dev_encodings_normal[contrastive_filter] @ contrastive_projection
#         z_h_row = dev_encodings_highlight[contrastive_filter] @ contrastive_projection
        z_nh_row = dev_encodings_irrelevant[contrastive_filter_nh] @ contrastive_projection_nh
#         z_all_nul = dev_encodings_normal[contrastive_filter] @ ambiguous_projection
#         z_h_nul = dev_encodings_highlight[contrastive_filter] @ ambiguous_projection
#         z_nh_nul = dev_encodings_irrelevant[contrastive_filter] @ ambiguous_projection

        clusters = (dev_preds_normal[contrastive_filter] == fact).astype(int)
#         clusters_h = clusters #(dev_preds_highlight[contrastive_filter] == fact).astype(int)
        clusters_nh = (dev_preds_irrelevant[contrastive_filter_nh] == fact).astype(int)

        cluster_distance_func = cluster_distance
        
        show_tsne = False
        
        n=1000
        if show_tsne:
            ig, axs = plt.subplots(2, 3, figsize=(15,10))
            ig.tight_layout()
        
        x, y = shuffle(z_all, clusters)
        dist_all = cluster_distance_func(x, y)
        if show_tsne:
            tsne_axis(x[:n], y[:n], ind2label=ind2label, 
                      title=f"alltext_normal; distance: {np.round(dist_all, 4)}", axs=axs[0, 0])
        
        x, y = shuffle(z_all_row, clusters)
        dist_all_row = cluster_distance_func(x, y)
        if show_tsne:
            tsne_axis(x[:n], y[:n], ind2label=ind2label, 
                      title=f"alltext_row; distance: {np.round(dist_all_row, 4)}", axs=axs[0, 1])
        
        x, y = shuffle(z_nh, clusters_nh)
        dist_nh = cluster_distance_func(x, y)
        if show_tsne:
            tsne_axis(x[:n], y[:n], ind2label=ind2label, 
                      title=f"del_normal; distance: {np.round(dist_nh, 4)}", axs=axs[1, 0])
        
        x, y = shuffle(z_nh_row, clusters_nh)
        dist_nh_row = cluster_distance_func(x, y)
        if show_tsne:
            tsne_axis(x[:n], y[:n], ind2label=ind2label, 
                      title=f"del_row; distance: {np.round(dist_nh_row, 4)}", axs=axs[1, 1])
            
#         x, y = shuffle(z_all_nul, clusters)
#         dist_all_nul = cluster_distance_func(x, y)
#         if show_tsne:
#             tsne_axis(x[:n], y[:n], ind2label=ind2label, 
#                       title=f"alltext_null; distance: {np.round(dist_all_nul, 4)}", axs=axs[0, 2])
        
#         x, y = shuffle(z_nh_nul, clusters_nh)
#         dist_nh_nul = cluster_distance_func(x, y)
#         if show_tsne:
#             tsne_axis(x[:n], y[:n], ind2label=ind2label, 
#                       title=f"del_null; distance: {np.round(dist_nh_nul, 4)}", axs=axs[1, 2])
        
        if show_tsne:
            plt.show()

#         print(pd.DataFrame([[dist_all, dist_all_row],[dist_nh, dist_nh_row]],
#                            index=["Normal space", "Rowspace"], columns=["All text", "Highlight is masked"]))
        
        print(f"Difference of distances:\t{np.round(dist_all - dist_nh, 4)}\t\t\t{np.round(dist_all_row - dist_nh_row, 4)}")#"\t\t\t{np.around(dist_all_nul - dist_nh_nul, 4)}")
#         print(dist_all_row)
#         print(dist_nh_row)
        diffs[-1].append(round(dist_all_row - dist_nh_row, 4))
        
diffs = np.array(diffs)

					architect vs professor


NameError: name 'cluster_distance' is not defined

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import re
from sklearn.metrics.pairwise import linear_kernel
# nltk.download('stopwords')
stopWords = stopwords.words('english')

texts = []
for c in index2label:
    text = ' '.join([' '.join(re.findall('(\\w+)', doc['text'].lower())) for doc in train_data if doc['label'] == index2label[c]])
    texts.append(text)
    
tfidf = TfidfVectorizer(stop_words = stopWords).fit_transform(texts)

cosine_similarities = linear_kernel(tfidf, tfidf)
# cosine_similarities

In [17]:
import numpy as np
import nullspace_projection.src.debias as debias
import pandas as pd

# rootdir = "/home/nlp/jacovia/ilp"  # location in biunlp
rootdir = "/home/alonj/contrastive"
dset = "dev"  # "train" "dev" "test"

# orig_encodings = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_train_encoded_representations.npy")
orig_encodings = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_encoded_representations.npy")
orig_labels = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_labels.npy")
orig_predictions = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_predictions.npy")

classifier_w = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/w.npy")
classifier_b = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/b.npy")

from scipy.spatial import distance
from scipy.stats import entropy
from scipy.special import softmax



def dkl(p,q):
    p = p / p.sum(axis=1).reshape(-1, 1)
    q = q / q.sum(axis=1).reshape(-1, 1)
    return entropy(p,q, axis=1) + entropy(q,p, axis=1)

for fact in index2label:
    drop = []
    print(f"\t\t\t\t\t{index2label[fact]}") # vs {index2label[foil]}")
    for foil in index2label:
        if fact == foil:
            drop.append(-1000)
            continue
            
#         directional_nullspace = debias.get_rowspace_projection((classifier_w[foil].reshape((1, -1))))
#         directional_nullspace = np.eye(directional_nullspace.shape[0]) - directional_nullspace

        # fact = 22
        # foil = 2
#         print(f"\t\t\t\t\t{index2label[fact]} vs {index2label[foil]}")
        
        predictions_del = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_highlight_predictions.npy")
        
#         print(distance.cosine(w[fact], w[foil]))
#         print("Cos-sim:", (w[fact] / np.linalg.norm(w[fact])).dot(w[foil] / np.linalg.norm(w[foil])) )
        
        
#         contrastive_examples = np.any([orig_predictions == fact, orig_predictions == foil], axis=0)
        contrastive_examples = np.any([orig_predictions == fact], axis=0)
#         print(contrastive_examples, contrastive_examples.sum())
#         contrastive_examples = np.any([orig_predictions == fact, predictions_del == foil, orig_labels == foil], axis=0)
#         contrastive_examples = np.all([orig_predictions == fact, predictions_del == foil], axis=0)
#         contrastive_examples = np.any([orig_predictions > -10000], axis=0)
    
        encodings = orig_encodings[contrastive_examples]  # only use examples of the fact/foil
        predictions = orig_predictions[contrastive_examples]
        
        if len(encodings) == 0:
            drop.append(0)
            continue

#         contrastive_rowspace_projection = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/projections/early/contrastive_{max(fact, foil)}_{min(fact, foil)}.npy")
#         contrastive_rowspace_projection = debias.get_rowspace_projection((classifier_w[fact] - classifier_w[foil]).reshape(1, -1))

        u = classifier_w[fact] - classifier_w[foil]
        contrastive_rowspace_projection = np.outer(u, u.T) / (u.dot(u.T))
        
        encodings_in_contrastive_rowspace = encodings @ contrastive_rowspace_projection #@ directional_nullspace


        assert (predictions == np.argmax(encodings @ classifier_w.T + classifier_b, axis=1)).all()

        predictions_in_contrastive_space = np.argmax(encodings_in_contrastive_rowspace @ classifier_w.T + classifier_b, axis=1)

#         print("Accuracy after contrastive rowspace projection: ",
#               sum(predictions_in_contrastive_space == predictions) / len(predictions))

        encodings_del = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_highlight_encoded_representations.npy")
        
        predictions_del = predictions_del[contrastive_examples]
        encodings_del = encodings_del[contrastive_examples]
        
#         contrastive_examples_nh = np.any([predictions_del == fact, predictions_del == foil], axis=0)
        
#         predictions_del = predictions_del[contrastive_examples_nh]
#         encodings_del = encodings_del[contrastive_examples_nh]
        
        assert (predictions_del == np.argmax(encodings_del @ classifier_w.T + classifier_b, axis=1)).all()

#         print("Accuracy after del: ",
#               sum(predictions_del == predictions) / len(predictions))
    
        encodings_both = encodings_del @ contrastive_rowspace_projection #@ directional_nullspace
        predictions_in_both = np.argmax(encodings_both @ classifier_w.T + classifier_b, axis=1)
#         print("Accuracy after both: ",
#               sum(predictions_in_both == predictions) / len(predictions))
#         print("Difference:", sum(predictions_in_contrastive_space == predictions) / len(predictions) - sum(predictions_in_both == predictions) / len(predictions))
        
#         drop.append(sum(predictions_in_contrastive_space == predictions) / len(predictions) - sum(predictions_in_both == predictions) / len(predictions))

        # encodings_in_amnesic_nullspace = encodings_in_contrastive_rowspace @ AMNESIC_PROJECTION
        # print("Accuracy after amnesic nullspace projection: ",
        #       sum(encodings_in_amnesic_nullspace == predictions) / len(predictions))
        
        prediction_probabilities = softmax(encodings @ classifier_w.T + classifier_b, axis=1)
        prediction_probabilities_contrastive = softmax(encodings_in_contrastive_rowspace @ classifier_w.T + classifier_b, axis=1)
        prediction_probabilities_both = softmax(encodings_both @ classifier_w.T + classifier_b, axis=1)
        prediction_probabilities_del = softmax(encodings_del @ classifier_w.T + classifier_b, axis=1)
        
        
#         prediction_probabilities = softmax(encodings @ classifier_w.T, axis=1)
#         prediction_probabilities_contrastive = softmax(encodings_in_contrastive_rowspace @ classifier_w.T, axis=1)
#         prediction_probabilities_both = softmax(encodings_both @ classifier_w.T, axis=1)
#         prediction_probabilities_del = softmax(encodings_del @ classifier_w.T, axis=1)
        
        dist1 = entropy(prediction_probabilities_contrastive, prediction_probabilities, axis=1).mean()
        dist2 = entropy(prediction_probabilities_both, prediction_probabilities, axis=1).mean()
        dist3 = entropy(prediction_probabilities_both, prediction_probabilities_contrastive, axis=1).mean()
        dist4 = entropy(prediction_probabilities_del, prediction_probabilities, axis=1).mean()
        
        dist3 = (entropy(prediction_probabilities_both, prediction_probabilities_contrastive, axis=1)
                + entropy(prediction_probabilities_contrastive, prediction_probabilities_both, axis=1)).mean()
        
        
        dist5 = prediction_probabilities_contrastive[:, fact] - prediction_probabilities_contrastive[:, foil]
        dist6 = prediction_probabilities_both[:, fact] - prediction_probabilities_both[:, foil]
        dist7 = (dist5 - dist6).mean()
        
        dist8 = dkl(prediction_probabilities_contrastive[:, [fact, foil]],
                    prediction_probabilities_both[:, [fact, foil]]).mean()
        
        p = prediction_probabilities_contrastive[:, [fact, foil]]
        q = prediction_probabilities_both[:, [fact, foil]]
#         p = prediction_probabilities[:, [fact, foil]]
#         q = prediction_probabilities_del[:, [fact, foil]]

        p = p / p.sum(axis=1).reshape(-1, 1)
        q = q / q.sum(axis=1).reshape(-1, 1)
        dist9 = (p[:, 0] - q[:, 0]).mean()
        
#         print("D_kl contrastive projection\t", dist1)
#         print("D_kl contrast+del projection\t", dist2)
#         print(len(dist5))
#         print("Diff\t", dist5.mean(), dist6.mean(), (dist5 - dist6).mean())
        drop.append(dist9)
    
    ratios = []
    for c in range(len(index2label)):
        genders = [doc['gender'] for doc in train_data if doc['label'] == index2label[c]]
        ratios.append(sum(g == 'm' for g in genders) / len(genders))

    tab = []
    print(fact)
    for c in range(len(index2label)):
        if c == fact:
            continue
        tab.append({
    #         'foil': index2label[c],
            'foil idx': c,
            'foil': index2label[c],
    #         'collapse': diffs[0][c],
            'measure': drop[c],
            'cos-sim': cosine_similarities[fact,c],
            'm ratio': ratios[c]
        })

    tab = sorted(tab, key=lambda v: -v['measure'])

    display(pd.DataFrame(tab))#, columns=['foil', 'collapse', 'cos-sim'])

#     print(f"& {tab[0]['foil']} & {tab[0]['measure']*100:.4f}")
    for row1,row2 in zip(tab[0:4],tab[-1:-4:-1]):
        print(f" & {row1['foil']} & {row1['measure']*100:.3f} & {row2['foil']} & {row2['measure']*100:.3f}")
    
#     print("==")
    
#     print(f"& {tab[-1]['foil']} & {tab[-1]['measure']*100:.4f}")
#     for row in tab[-1:-4:-1]:
#         print(f" & {row['foil']} & {row['measure']*100:.4f}")


					professor
0


,foil idx,foil,measure,cos-sim,m ratio
0,1,physician,0.001404,0.352405,0.475674
1,4,journalist,0.000630,0.506371,0.506323
2,14,software_engineer,0.000320,0.503450,0.843354
3,12,poet,0.000255,0.343901,0.511597
4,8,surgeon,0.000238,0.450460,0.848932
5,3,photographer,0.000223,0.317630,0.651584
6,16,accountant,0.000141,0.380589,0.636141
7,19,pastor,0.000132,0.339296,0.756189
8,13,filmmaker,0.000131,0.285122,0.664169
9,7,teacher,0.000127,0.573837,0.398395


 & physician & 0.140 & psychologist & -0.032
 & journalist & 0.063 & architect & -0.021
 & software_engineer & 0.032 & attorney & -0.018
					physician
1


,foil idx,foil,measure,cos-sim,m ratio
0,0,professor,0.006223,0.352405,0.550123
1,8,surgeon,0.004061,0.712523,0.848932
2,6,psychologist,0.001847,0.405883,0.381618
3,4,journalist,0.001844,0.186806,0.506323
4,2,attorney,0.001284,0.229435,0.621217
5,7,teacher,0.001216,0.251159,0.398395
6,17,dietitian,0.000727,0.273999,0.072343
7,16,accountant,0.000603,0.165945,0.636141
8,3,photographer,0.000578,0.121858,0.651584
9,14,software_engineer,0.000554,0.153514,0.843354


 & professor & 0.622 & nurse & -0.315
 & surgeon & 0.406 & chiropractor & -0.132
 & psychologist & 0.185 & paralegal & -0.013
					attorney
2


...

 & architect & 4.095 & composer & 0.081
 & photographer & 2.946 & chiropractor & 0.093
 & journalist & 2.046 & paralegal & 0.105
					personal_trainer
25


,foil idx,foil,measure,cos-sim,m ratio
0,4,journalist,0.014848,0.346216,0.506323
1,3,photographer,0.014589,0.285202,0.651584
2,7,teacher,0.013463,0.406775,0.398395
3,0,professor,0.012180,0.280896,0.550123
4,2,attorney,0.012106,0.255768,0.621217
5,6,psychologist,0.009560,0.352226,0.381618
6,5,nurse,0.009540,0.175477,0.089235
7,12,poet,0.008731,0.194307,0.511597
8,17,dietitian,0.008457,0.399158,0.072343
9,16,accountant,0.008414,0.299345,0.636141


 & journalist & 1.485 & paralegal & 0.240
 & photographer & 1.459 & chiropractor & 0.244
 & teacher & 1.346 & composer & 0.297
					rapper
26


,foil idx,foil,measure,cos-sim,m ratio
0,23,dj,0.018778,0.538838,0.853129
1,15,composer,0.018440,0.462786,0.839506
2,12,poet,0.013348,0.243766,0.511597
3,7,teacher,0.013218,0.306494,0.398395
4,4,journalist,0.010532,0.322975,0.506323
5,3,photographer,0.010244,0.276814,0.651584
6,18,comedian,0.008685,0.269224,0.786096
7,0,professor,0.007422,0.165420,0.550123
8,1,physician,0.007358,0.069247,0.475674
9,6,psychologist,0.007348,0.173120,0.381618


 & dj & 1.878 & paralegal & 0.130
 & composer & 1.844 & dietitian & 0.177
 & poet & 1.335 & interior_designer & 0.272


In [20]:
import pandas as pd

ratios = []
for c in range(len(index2label)):
    genders = [doc['gender'] for doc in train_data if doc['label'] == index2label[c]]
    ratios.append(sum(g == 'm' for g in genders) / len(genders))

tab = []
print(fact)
for c in range(len(index2label)):
    if c == fact:
        continue
    tab.append({
#         'foil': index2label[c],
        'foil idx': c,
        'foil': index2label[c],
#         'collapse': diffs[0][c],
        'measure': drop[c],
        'cos-sim': cosine_similarities[fact,c],
        'm ratio': ratios[c]
    })

tab = sorted(tab, key=lambda v: -v['measure'])
    
pd.DataFrame(tab)#, columns=['foil', 'collapse', 'cos-sim'])




22


,foil idx,foil,measure,cos-sim,m ratio
0,2,attorney,0.022081,0.695623,0.621217
1,16,accountant,0.009358,0.403303,0.636141
2,23,dj,0.007016,0.138354,0.853129
3,19,pastor,0.006013,0.245757,0.756189
4,25,personal_trainer,0.005982,0.249215,0.548564
5,26,rapper,0.004069,0.141270,0.903315
6,18,comedian,0.003146,0.121218,0.786096
7,7,teacher,0.002505,0.354448,0.398395
8,6,psychologist,0.001835,0.294871,0.381618
9,5,nurse,0.001824,0.173373,0.089235


In [21]:
from IPython.display import display

In [23]:
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression

X = np.array([[v['cos-sim']] for v in tab])
Y = np.array([v['measure'] for v in tab])
reg = LinearRegression().fit(X, Y)
print('lexical similarity -> dkl\t\t', reg.score(X, Y))

X = np.array([[v['m ratio']] for v in tab])
# Y = np.array([v['dkl drop'] for v in tab])
reg = LinearRegression().fit(X, Y)
print('m ratio -> dkl\t\t\t\t', reg.score(X, Y))

X = np.array([[v['cos-sim'],v['m ratio']] for v in tab])
# Y = np.array([v['dkl drop'] for v in tab])
reg = LinearRegression().fit(X, Y)
print('lexical similarity + m ratio -> dkl\t', reg.score(X, Y))

lexical similarity -> dkl		 0.5320527099142018
m ratio -> dkl				 0.03071002733385375
lexical similarity + m ratio -> dkl	 0.5633119801734481


In [127]:
import numpy as np
import statsmodels.api as sm

X = np.array([[v['cos-sim']] for v in tab])
X = sm.add_constant(X)
y = np.array([v['measure'] for v in tab])
results = sm.OLS(y, X).fit()
print(results.summary())

X = np.array([[v['m ratio']] for v in tab])
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

X = np.array([[v['cos-sim'],v['m ratio']] for v in tab])
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     36.34
Date:                Thu, 28 Jan 2021   Prob (F-statistic):           3.18e-06
Time:                        16:37:25   Log-Likelihood:                 115.61
No. Observations:                  26   AIC:                            -227.2
Df Residuals:                      24   BIC:                            -224.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0029      0.001     -2.114      0.0

In [56]:
diffsnp = np.array(diffs[0])
print([index2label[l] for l in np.argsort(diffsnp)][1:])

['physician', 'professor', 'attorney', 'photographer', 'journalist', 'psychologist', 'teacher', 'surgeon', 'dentist', 'poet', 'architect', 'painter', 'filmmaker', 'software_engineer', 'accountant', 'composer', 'dietitian', 'chiropractor', 'pastor', 'comedian', 'yoga_teacher', 'personal_trainer', 'rapper', 'interior_designer', 'paralegal', 'dj']


In [13]:
diffsnp = np.array(diffs[5])
# print(diffsnp)
# print(index2label[np.argmin(diffsnp)])
print([index2label[l] for l in np.argsort(diffsnp)])

['yoga_teacher', 'nurse', 'personal_trainer', 'poet', 'painter', 'photographer', 'interior_designer', 'attorney', 'paralegal', 'dj', 'dietitian', 'filmmaker', 'journalist', 'teacher', 'rapper', 'comedian', 'pastor', 'composer', 'accountant', 'software_engineer', 'psychologist', 'architect', 'physician', 'chiropractor', 'professor', 'dentist', 'surgeon']


In [10]:
for fact in index2label:
    diffsnp = np.array(diffs[fact])
    # print(diffsnp)
    # print(index2label[np.argmin(diffsnp)])
    print(index2label[fact], '\t\t', [index2label[l] for l in np.argsort(-diffsnp)[:5]])

professor 		 ['physician', 'paralegal', 'attorney', 'nurse', 'dietitian']
physician 		 ['surgeon', 'professor', 'dentist', 'nurse', 'psychologist']
attorney 		 ['paralegal', 'professor', 'architect', 'accountant', 'psychologist']
photographer 		 ['interior_designer', 'paralegal', 'painter', 'filmmaker', 'composer']
journalist 		 ['teacher', 'filmmaker', 'paralegal', 'composer', 'professor']
nurse 		 ['surgeon', 'dentist', 'professor', 'physician', 'chiropractor']
psychologist 		 ['professor', 'chiropractor', 'teacher', 'surgeon', 'nurse']
teacher 		 ['journalist', 'professor', 'filmmaker', 'architect', 'psychologist']
surgeon 		 ['physician', 'nurse', 'chiropractor', 'dietitian', 'dentist']
dentist 		 ['nurse', 'chiropractor', 'physician', 'surgeon', 'dietitian']
architect 		 ['interior_designer', 'paralegal', 'professor', 'nurse', 'attorney']
painter 		 ['photographer', 'filmmaker', 'composer', 'interior_designer', 'journalist']
poet 		 ['composer', 'filmmaker', 'paralegal', 'interior

In [12]:
for fact in index2label:
    diffsnp = np.array(diffs[fact])
    # print(diffsnp)
    # print(index2label[np.argmin(diffsnp)])
    print(index2label[fact], '\t\t', [index2label[l] for l in np.argsort(-diffsnp)[:5]])

professor 		 ['physician', 'paralegal', 'nurse', 'attorney', 'dietitian']
physician 		 ['surgeon', 'professor', 'dentist', 'nurse', 'psychologist']
attorney 		 ['paralegal', 'professor', 'architect', 'psychologist', 'accountant']
photographer 		 ['interior_designer', 'paralegal', 'painter', 'filmmaker', 'rapper']
journalist 		 ['paralegal', 'teacher', 'filmmaker', 'professor', 'composer']
nurse 		 ['surgeon', 'dentist', 'professor', 'chiropractor', 'physician']
psychologist 		 ['professor', 'chiropractor', 'teacher', 'paralegal', 'nurse']
teacher 		 ['journalist', 'professor', 'filmmaker', 'architect', 'psychologist']
surgeon 		 ['physician', 'nurse', 'chiropractor', 'dietitian', 'dentist']
dentist 		 ['nurse', 'chiropractor', 'physician', 'surgeon', 'dietitian']
architect 		 ['interior_designer', 'professor', 'paralegal', 'nurse', 'teacher']
painter 		 ['photographer', 'filmmaker', 'composer', 'paralegal', 'interior_designer']
poet 		 ['composer', 'paralegal', 'interior_designer', 'fi

In [4]:
import pickle
with open("../data/bios/dev.pickle", "rb") as f:
    data = pickle.load(f)

print([ex for ex in data if ex['p'] == 'model'][15:50])
    
yy = 2700
for a in data[yy:yy+20]:
#     if a['p'] == 'attorney':
        print(json.dumps({'text': a['hard_text'], 'label': a['p']}))
#         break

[{'g': 'f', 'p': 'model', 'text': 'Eva Jude is a Adult Model from Ukraine. She was born on January 1, 2000. She is listed on FreeOnes since 2018 and is currently ranked 12450th place. She currently has 5 gallery links in her own FreeOnes section.', 'start': 39, 'hard_text': 'She was born on January 1, 2000. She is listed on FreeOnes since 2018 and is currently ranked 12450th place. She currently has 5 gallery links in her own FreeOnes section.', 'text_without_gender': '_ was born on January 1, 2000. _ is listed on FreeOnes since 2018 and is currently ranked 12450th place. _ currently has 5 gallery links in _ own FreeOnes section.'}, {'g': 'f', 'p': 'model', 'text': 'Pam Kiss is a German model with really nice and big breasts. In 2014 she got a Netstar Award. This photorealistic 3D printed figurine is made with a full-color 3D printer and received an elaborate finishing touch by hand. The figurine is made of full color sandstone It comes in a lavish gift box that includes a Certificate 

NameError: name 'json' is not defined